## Segmenting and Clustering Neighborhoods in Toronto by Fei

### Part 1  
#### Obtain the data "List of postal codes of Canada: M" from Wikipedia page, and transform the data into a pandas dataframe.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# load data into dataframe
dataframe = pd.read_html(url)
dataframe = dataframe[0]
dataframe.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [4]:
# rename column names
dataframe.columns = ['PostalCode', 'Borough', 'Neighborhood']
dataframe.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [5]:
# Ignore cells with a borough that is Not assigned.
assigned_dataframe = dataframe[dataframe['Borough']!='Not assigned']
# reset index
assigned_dataframe.reset_index(inplace=True, drop=True)
assigned_dataframe.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
# Check if more than one neighborhood exist in one postal code area.
print("assigned_dataframe shape: ", assigned_dataframe.shape)
print("unique PostalCode: ", len(assigned_dataframe['PostalCode'].unique()))

assigned_dataframe shape:  (103, 3)
unique PostalCode:  103


No duplicated PostalCode exists.

In [7]:
# Check if a cell has a borough but a Not assigned neighborhood
assigned_dataframe[assigned_dataframe['Neighborhood']=='Not assigned'].count()

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

There's no "Not assigned" neighborhood.

In [8]:
assigned_dataframe.shape

(103, 3)

### Part 2  
#### I wasn't able to install the Geocoder package, so I will use the csv file that has the geographical coordinates of each postal code.

In [9]:
# download the data
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')
# read it into a pandas dataframe.
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.head(10)

Data downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [10]:
# rename coordinates_df columns
coordinates_df.columns = ['PostalCode', 'Latitude', 'Longitude']
coordinates_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# merge assigned_dataframe with coordinates_df to add latitude/longitude for each neighborhood
merged_dataframe = assigned_dataframe
merged_dataframe = merged_dataframe.join(coordinates_df.set_index('PostalCode'), on='PostalCode')
merged_dataframe.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Part 3  
### Explore and cluster the neighborhoods in Toronto.

#### Select only boroughs that contain the word Toronto

In [12]:
# select only boroughs that contain the word Toronto
toronto_df = merged_dataframe[merged_dataframe['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


#### I wasn't able to install geocoder package, so I got the coordinates of Toronto, Ontario from google manually.

In [13]:
# get the geographical coordinates of Toronto, Ontario.
latitude = 43.6532
longitude = -79.3832
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6532, -79.3832.


Import libraries

In [14]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# !conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

#### Create a map of Toronto with neighborhoods superimposed on top.

In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)  
map_toronto

### Explore the neighborhoods in Toronto.  
#### Utilizing the Foursquare API to explore the neighborhoods and segment them.

In [16]:
# Define Foursquare Credentials and Version
CLIENT_ID = '5RR4BF503NMA5Z3T1BVQDUFYH4D4KRESSO4TE1MCGVIUF4QC'
CLIENT_SECRET = '4KA5SMNYGO4GC5GE0QI4ROQRPMMFU5LS541FZ0NQWEH2Y5UB' 
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

#### Create a function to repeat the same process to all the neighborhoods in Toronto.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)  
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']       
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [18]:
# run the above function on each neighborhood and create a new dataframe called toronto_venues.
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'], latitudes=toronto_df['Latitude'], longitudes=toronto_df['Longitude'])
# check the size of the resulting dataframe
print(toronto_venues.shape)
print(toronto_venues.head())

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Analyze Each Neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.head())
print(toronto_onehot.shape)

   Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Theme Restaurant  Toy / Game Store

In [20]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.head())
print(toronto_grouped.shape)

                                        Neighborhood  Yoga Studio  \
0                                        Berczy Park     0.000000   
1       Brockton, Parkdale Village, Exhibition Place     0.041667   
2  Business reply mail Processing Centre, South C...     0.062500   
3  CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                 Central Bay Street     0.015385   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                0.0   0.0000              0.0000        0.0000   
1                0.0   0.0000              0.0000        0.0000   
2                0.0   0.0000              0.0000        0.0000   
3                0.0   0.0625              0.0625        0.0625   
4                0.0   0.0000              0.0000        0.0000   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0           0.000           0.0000             0.000                  0.0   
1           0.000           

#### Display the top 10 venues for each neighborhood.

In [21]:
# write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
print(neighborhoods_venues_sorted.head())

                                        Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...           Yoga Studio   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0        Farmers Market                Bakery                  Café   
1           Coffee Shop                Bakery        Breakfast Spot   
2         Auto Workshop                  Park            Comic Shop   
3      Airport Terminal        Airport Lounge           Coffee Shop   
4        Sandwich Place    Italian Restaurant                  Café   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant          

###  Cluster the neighborhoods in Toronto.  
#### Run k-means to cluster the neighborhood into 5 clusters.

In [23]:
# import KMeans
from sklearn.cluster import KMeans

In [24]:
# set number of clusters
kclusters = 5
toronto_clustering = toronto_df.drop(['PostalCode','Borough','Neighborhood'], 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1], dtype=int32)

In [25]:
# add clustering labels to toronto_df.
toronto_merged = toronto_df
toronto_merged.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged.head(10)

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,3,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,1,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


#### Visualize the resulting clusters.

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
       
map_clusters